In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup, SoupStrainer
import numpy as np
import re
import matplotlib.pyplot as plt

import pandas as pd
from tqdm.auto import tqdm
from lxml.html import fromstring
from itertools import cycle

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random

from time import sleep

from skimage import io

from concurrent.futures import ThreadPoolExecutor

import json
import pickle

In [2]:
def get_soup_simple(url, strain_name=None, strain_attrs=None):
    successful = False
    while not successful:
        try:
            response = requests.get(url)#, headers=headers, proxies=proxies)
            if strain_name is None and strain_attrs is None:
                soup = BeautifulSoup(response.text, 'lxml')
            elif strain_attrs is None:
                strainer = SoupStrainer(strain_name)
                soup = BeautifulSoup(response.text, 'lxml', parse_only=strainer)
            else:
                strainer = SoupStrainer(strain_name, strain_attrs)
                soup = BeautifulSoup(response.text, 'lxml', parse_only=strainer)
            successful = True
        except BaseException as e:
            continue
    return soup

In [54]:
base_url = 'https://araneae.nmbe.ch/'
families_url = base_url + 'list/families'

main_page = get_soup_simple(families_url, 'tbody')

In [100]:
df = pd.read_json('data/spiderbase_1.json')

In [101]:
def group_images(img_content, species):
    groups = {'habitus': [], 'epigyne': [], 'pedipalp': [], 'vulve': [], 'other': []}
    for img in img_content:
        name = img['name'].lower()
        if 'epigyne' in name:
            groups['epigyne'].append(img)
        elif 'pedipalp' in name:
            groups['pedipalp'].append(img)
        elif 'vulve' in name:
            groups['vulve'].append(img)
        elif 'habitus' in name:
            groups['habitus'].append(img)
        else:
            groups['other'].append(img)
    return groups
    
def get_image_metainfo_quick(args):
    url, species_name = args
    img_content = []
    sleep(0.1)
    try:
        img_data = get_soup_simple(url, 'div', {'class': 'thumbnail'})('div', 'thumbnail')
        for dt in img_data:
            sleep(0.1)
            name = dt.p.text.lstrip()
            credits = dt.a['title']
            link = base_url+dt.img['src']
            image = io.imread(link)
            img_content.append({'name': name, 'image': image , 'credits': credits})

        groups = group_images(img_content, species_name)
        with open(f'data/image_data_1/{species_name.replace(" ","-").lower()}.pkl','wb') as out_file:
            pickle.dump(groups, out_file)
        return True, args, ''
    except Exception as e:
        return False, args, e

In [102]:
arg_list= df[['link', 'species']].values.astype(str)

results = []
for arg in tqdm(arg_list):
    results.append(get_image_metainfo_quick(arg))